In [ ]:
import requests
from bs4 import BeautifulSoup, NavigableString
import csv


url = 'https://issues.apache.org/jira/browse/CAMEL-10597'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
type_ = soup.find('span', {'class': 'value', 'id': 'type-val'}).get_text(strip=True)
status = soup.find('span', {'class': 'value', 'id': 'status-val'}).get_text(strip=True)
priority = soup.find('span', {'class': 'value', 'id': 'priority-val'}).get_text(strip=True)
resolution = soup.find('span', {'class': 'value', 'id': 'resolution-val'}).get_text(strip=True)
versions = soup.find('span', {'class': 'value', 'id': 'versions-val'}).get_text(strip=True)
fix = soup.find('span', {'class': 'value', 'id': 'fixfor-val'}).get_text(strip=True)
components = soup.find('span', {'class': 'value', 'id': 'components-val'}).get_text(strip=True)
labels = soup.find('div', {'class': 'labels-wrap value'}).get_text(strip=True)
patch = soup.find('div', {'class': 'value type-multicheckboxes', 'id': 'customfield_12310041-val'}).get_text(strip=True)
complexity = soup.find('div', {'class': 'value type-select', 'id': 'customfield_12310060-val'}).get_text(strip=True)
assignee = soup.find('span', {'class': 'view-issue-field', 'id': 'assignee-val'}).get_text(strip=True)
reporter = soup.find('span', {'class': 'view-issue-field', 'id': 'reporter-val'}).get_text(strip=True)
created = soup.find('time', {'class': 'livestamp'})['datetime']
description = soup.find('div',{'class':'user-content-block'}).get_text()

comments1 = str(soup.find_all('script', attrs={'id': False, 'type': False})[3].get_text())
comments2 = comments1.replace('\\u003e', '>').replace('\\u003c', '<').replace('\\\\\\"','"').replace('\\\\n','\n').replace('\\\\/','/').replace('\\\'','\'')

comments3 = BeautifulSoup(comments2, 'html.parser')
comments4 = comments3.find_all(['a','span'])
users_ = []
times_ = []
comments6 = []
comments = []
comments5 = comments3.find_all('div',attrs={'class':'action-details'})
for comment in comments5:
    for child in comment.children:
        if isinstance(child, NavigableString):
              comments6.append(child.get_text(strip=True))

for comment in comments6:
    if comment != 'added a comment  -' and comment != '':
        comments.append(comment)

for comment in comments4:
    if comment.name == 'a' and len(comment.get('class',[''])) == 2 and comment.get('class',[''])[1] == 'user-avatar':
        users_.append(comment.get_text(strip=True))
    if comment.name == 'span' and comment.get('class',[''])[0] == 'date':
        times_.append(comment.get_text())

users = users_[::2]
times = times_[::2]

filename = "output.csv"
Ln1 = ["type","status","priority","resolution","versions","fix_version","components","labels","patch","estimated_complexity","assignee","reporter","created","description","comments"]
comments_total = ""
for n in range(len(comments)):
    comments_total = comments_total+users[n]+": "+times[n]+": \""+comments[n]+"\"; "
Ln2 = [type_,status,priority,resolution,versions,fix,components,labels,patch,complexity,assignee,reporter,created,description,comments_total]
data = [Ln1,Ln2]
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)